# Model Name: Neural Collaborative Based Filtering

- using neural network to learn the user-item interaction

This piece is a TensorFlow implementation of Neural Collaborative Filtering (NCF) from the paper [He et al. (2017)](https://arxiv.org/pdf/1708.05031.pdf).

Summary: NCF uses neural networks to model the interactions between users and items. NCF replaces the inner product (used in ordinary MF methods) with a neural architecture that can learn an arbitrary function from data. This allows NCF to express and generalize matrix factorization under its framework. Essentially, it uses a neural network to learn the user-item interaction function, and uses the learned function to predict the corresponding rating. A multi-layer perceptron is used to learn the user-item interaction function. 

## Preliminaries

In [3]:
# reset space
%reset -f

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

## Model Building

## Evaluation


***
# Example

This is a basic implementation of the NCF model. For more advanced features (like adding more layers to the model or using different activation functions), you might need to modify the code accordingly. Also, remember to handle overfitting and underfitting by tuning your model and using techniques like early stopping, regularization, etc.

The code below will train a NCF model on your data and then use it to predict the ratings. The predicted ratings will be stored in the y_pred variable. You can adjust the parameters of the model (such as the number of epochs, the batch size, and the dimensions of the embedding layers) to better fit your data

The steps are as follows:





In [14]:
# reset space
%reset -f

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# tensorflow libraries
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [15]:
x = pd.read_csv(r"C:\Users\e1002902\Documents\GitHub Repository\Masters-Dissertation\Code\temp_data.csv")

# Melt the DataFrame into a format where each row is a user-item interaction
x_melt = x.melt(id_vars=x.columns[0], var_name='book', value_name='rating')

# rename columns
x_melt.columns = ['user', 'book', 'rating']

# Filter out the rows where rating is 0
x_melt = x_melt[x_melt['rating'] != 0]
x_melt

# Convert user and book to categorical
x_melt['user'] = x_melt['user'].astype('category')
x_melt['book'] = x_melt['book'].astype('category')


In [23]:
# Create user and book embedding layers
user_input = Input(shape=(1,), dtype='int32', name='user_input')
book_input = Input(shape=(1,), dtype='int32', name='book_input')

user_embedding = Embedding(input_dim=len(x_melt['user'].cat.categories), output_dim=50, name='user_embedding')(user_input)
book_embedding = Embedding(input_dim=len(x_melt['book'].cat.categories), output_dim=50, name='book_embedding')(book_input)

# Flatten the embedding vectors
user_vecs = Flatten()(user_embedding)
book_vecs = Flatten()(book_embedding)

# Concatenate the embedding vectors
input_vecs = Concatenate()([user_vecs, book_vecs])

# Add dense layers
x = Dense(128, activation='relu')(input_vecs)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
y = Dense(1)(x)

# Compile the model
model = Model(inputs=[user_input, book_input], outputs=y)
model.compile(optimizer=Adam(0.001), loss='mse')

# Prepare the data
X = x_melt[['user', 'book']].apply(lambda x: x.cat.codes)
y = x_melt['rating']

# Normalize ratings to be between 0 and 1
y = (y - 1) / 4

In [30]:
# Train the model
model.fit([X['user'], X['book']], y, batch_size=64, epochs=50, validation_split=0.1)

# Predict the ratings
y_pred = model.predict([X['user'], X['book']])

# Rescale the predictions back to the 1-5 range
y_pred = y_pred * 4 + 1



Epoch 1/50
1/1 [==============================] - 0s 443ms/step - loss: 0.2665 - val_loss: 0.2964
Epoch 2/50
1/1 [==============================] - 0s 211ms/step - loss: 0.2450 - val_loss: 0.2657
Epoch 3/50
1/1 [==============================] - 0s 139ms/step - loss: 0.2045 - val_loss: 0.2354
Epoch 4/50
1/1 [==============================] - 0s 266ms/step - loss: 0.1827 - val_loss: 0.2058
Epoch 5/50
1/1 [==============================] - 0s 243ms/step - loss: 0.1603 - val_loss: 0.1771
Epoch 6/50
1/1 [==============================] - 0s 249ms/step - loss: 0.1460 - val_loss: 0.1497
Epoch 7/50
1/1 [==============================] - 0s 185ms/step - loss: 0.1221 - val_loss: 0.1244
Epoch 8/50
1/1 [==============================] - 0s 235ms/step - loss: 0.0843 - val_loss: 0.1017
Epoch 9/50
1/1 [==============================] - 0s 216ms/step - loss: 0.0952 - val_loss: 0.0824
Epoch 10/50
1/1 [==============================] - 0s 212ms/step - loss: 0.0832 - val_loss: 0.0669
Epoch 11/50
1/1 [==

In [32]:
pd.DataFrame(y_pred).describe()

,0
count,68.000000
mean,3.665163
std,0.640481
min,2.352252
25%,3.186118
50%,3.680455
75%,4.064956
max,4.895488


### Experiementing

Experimenting with different parameters and visualizing the training process can provide valuable insights into how well our model is performing and where improvements can be made.

- ***Plot Training and Validation Loss:*** This can help you understand if your model is overfitting or underfitting. If your training loss is much lower than your validation loss, your model might be overfitting. If both losses are high, your model might be underfitting.

- ***Experiment with Different Architectures:*** Try adding more layers to your model or increasing the number of neurons in each layer. You could also experiment with different types of layers (e.g., convolutional layers, recurrent layers) and different activation functions.

- ***Tune Hyperparameters***: This includes the learning rate, batch size, number of epochs, and regularization parameters. You could use techniques like grid search or random search to systematically explore different combinations of hyperparameters.

- ***Use Early Stopping:*** This technique allows you to stop training once the model’s performance on a validation set stops improving, which can be useful to prevent overfitting.

- ***Try Different Optimization Algorithms:*** In addition to Adam, there are many other optimization algorithms available in TensorFlow, such as SGD, RMSprop, and Adagrad. Different optimizers might lead to different results.

- ***Regularization:*** If your model is overfitting, you might want to add some form of regularization, such as L1 or L2 regularization, or dropout.

- ***Data Augmentation***: If you have a small dataset, you could artificially increase its size by creating modified versions of your existing data. For example, you could add small amounts of noise to your input data.

- ***Learning Rate Scheduling:*** Instead of using a fixed learning rate, you could decrease it over time or in response to the model’s performance.


In [ ]:
history = model.fit([X_train['user'], X_train['book']], y_train, batch_size=64, epochs=5, validation_split=0.1)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
